In [1]:
#LIBRERIAS DEL PROYECTO

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

#Este es driver que va a navegar.
driver = webdriver.Chrome()

# Función para iniciar sesión
arancia_url = 'http://arancia-001-site1.btempurl.com/'

driver.get(arancia_url)
time.sleep(3)

# Entrar a la web
element = driver.find_element(By.ID,"Button1")
element.click()
time.sleep(3)

element = driver.find_element(By.ID, 'TextBox1')
element.send_keys('FRANCESC')
element = driver.find_element(By.ID, 'TextBox2')
element.send_keys('n4r4nj4720')

element = driver.find_element(By.ID,"Button1")
element.click()
time.sleep(3)

element = driver.find_element(By.ID,"Button10")
element.click()
time.sleep(3)

driver.switch_to.frame('facturacion')
element = driver.find_element(By.ID,"Button3")
element.click()

In [1]:
import pandas as pd
import string
import numpy as np

data = pd.read_excel(r'C:\Users\ThinkPad-PC\Downloads\Mis Comprobantes Recibidos - CUIT 30714894346.xlsx', header=0, skiprows=1)

#TIPO DE FACTURA
data['Tipo'] = data['Tipo'].astype('string').str[-9:]
data['Tipo2'] = data['Tipo'].str[:7]
data['Tipo3'] = data['Tipo'].str[-1:]

# FACTURA
data['Punto de Venta'] = data['Punto de Venta'].astype(str).str.zfill(5)
data['Número Desde'] = data['Número Desde'].astype(str).str.zfill(8)
data['Factura'] = data['Punto de Venta'] + '-' + data['Número Desde']

# PROVEEDOR
data['Denominación Emisor'] = data['Denominación Emisor'].str.translate(
    str.maketrans('', '', string.punctuation))
data['Proveedor'] = data['Denominación Emisor'].str[:35]

# CUIT
data['CUIT'] = data['Nro. Doc. Emisor'].astype(str)


# IMPORTES
data['NETO 10.5'] = data['Neto Grav. IVA 10,5%'] 
data['IVA 10.5'] = data['IVA 10,5%'] 
data['NETO 21'] = data['Neto Grav. IVA 21%'] 
data['IVA 21'] = data['IVA 21%'] 
data['NO GRAVADO'] = data['Neto No Gravado'] 
data['EXENTO'] = data['Op. Exentas'] 
data['IMPUESTOS'] = data['Otros Tributos']
data['NETO 0'] = data['Neto Grav. IVA 0%']
data['TOTAL_NO_GRAVADO'] = (data['NO GRAVADO'] + data['EXENTO'] + data['IMPUESTOS'] + data['NETO 0'])

# PONER 0 A TODAS LAS COLUMNAS
columnas = ['NETO 0', 'NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS',
            'IVA 2,5%', 'IVA 5%', 'IVA 27%', 'TOTAL_NO_GRAVADO'] 
for col in columnas: 
    data[col] = data[col].fillna(0)

# TIPO DE CAMBIO ESTE CAMBIAR PORQUE CAMBIA A FUTURO
columnas_tc = ['NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS'] 
for col in columnas_tc: 
    data[col] = data[col] * data['Tipo Cambio']

# SUMAR LO QUE NO LLEVA IVA
data['TOTAL_NO_GRAVADO'] = (data['NO GRAVADO'] + data['EXENTO'] + data['IMPUESTOS'] + data['NETO 0'])

# Convertimos las columnas a str
data['NETO 10.5'] = data['NETO 10.5'].astype(str)
data['IVA 10.5'] = data['IVA 10.5'].astype(str)
data['NETO 21'] = data['NETO 21'].astype(str)
data['IVA 21'] = data['IVA 21'].astype(str)

# PONER EN NEGATICO LAS COLUMNAS QUE SON CREDITO
for col in columnas:
    data.loc[data['Tipo2'] == 'Crédito', col] = -data[col].astype(float)
    data[col] = data[col].astype(str)

# CREAR Y APLICAR MÁSCARA
clean = ['Fecha','Tipo2','Tipo3', 'Factura',
        'Proveedor','CUIT', 'NETO 10.5', 
        'NETO 21', 'IVA 10.5', 'IVA 21', 
        'TOTAL_NO_GRAVADO']

clean_data = data[clean]
clean_data.head(5)

,Fecha,Tipo2,Tipo3,Factura,Proveedor,CUIT,NETO 10.5,NETO 21,IVA 10.5,IVA 21,TOTAL_NO_GRAVADO
0,01/10/2025,Factura,A,00009-00011663,INVERSIONES MAIPO S A,30699533412,0.0,96600.0,0.0,20286.0,0.0
1,01/10/2025,Factura,A,00008-00089201,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,474597.45
2,01/10/2025,Factura,A,00008-00089208,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,1161272.0
3,01/10/2025,Factura,A,00003-00017721,FIGUEROA EDGARDO DANIEL,20165212577,0.0,32931.36,0.0,6911.5199999999995,273455.16000000003
4,01/10/2025,Factura,A,00008-00089213,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,140000.0


In [2]:
from afip_data_transformation import transform_afip_inbound_invoices # -> Transforma los comprobantes en formato que necesitamos
from bookit_data_transformation import procesar_inbound_html # -> Transforma el registro de afip
from data_comparison import comparar_facturas_compra # -> Compara ambos registros y devuelve una lista de los que no están actualizados

In [3]:
import os
import lxml
import pandas as pd
import string
import numpy as np

afip_data = pd.read_excel('../downloads/recibidos_octubre.xlsx', header=0, skiprows=1)

afip_data_inbound = transform_afip_inbound_invoices(afip_data)
bookit_data_inbound = procesar_inbound_html('../inbound.html')

✅ Archivo ../inbound.html procesado correctamente (305 filas).


In [4]:

clean_data = comparar_facturas_compra(afip_data_inbound,bookit_data_inbound,1)

⚠️  94 facturas no están cargadas en el sistema.
❌ La factura 00008-00007656 no está cargada correctamente (diferencia en totales).
✅ 94 facturas correctamente cargadas y listas para procesar.


c:\Users\ThinkPad-PC\Documents\GitHub\AFIP-nuevo\src\data_comparison.py:123: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  merged.loc[~merged['loaded'], 'totales_ok'] = np.nan
c:\Users\ThinkPad-PC\Documents\GitHub\AFIP-nuevo\src\data_comparison.py:127: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mal_cargadas = merged[(merged['loaded']) & (~merged['totales_ok'].fillna(False))]


In [5]:
clean_data

,Fecha,Tipo2,Tipo3,Factura,Proveedor,CUIT,NETO 10.5,NETO 21,IVA 10.5,IVA 21,TOTAL_NO_GRAVADO
174,16/10/2025,Factura,A,00008-00092115,CARNIVAL SRL,30676579938,0.0,0.00,0.00,0.00,2142035.00
207,20/10/2025,Factura,A,00008-00092501,CARNIVAL SRL,30676579938,0.0,0.00,0.00,0.00,31164685.00
209,20/10/2025,Crédito,A,00008-00007951,CARNIVAL SRL,30676579938,-0.0,-0.00,-0.00,-0.00,-382500.00
210,20/10/2025,Crédito,A,00008-00007952,CARNIVAL SRL,30676579938,-0.0,-0.00,-0.00,-0.00,-4766580.00
307,27/10/2025,Factura,A,00008-00094003,CARNIVAL SRL,30676579938,1336810.1,0.00,140365.06,0.00,28635.46
...,...,...,...,...,...,...,...,...,...,...,...
394,31/10/2025,Crédito,A,00003-00000124,HERNANDEZ FRANCISCO GUSTAVO Y ROSSI,30707654348,-0.0,-339917.36,-0.00,-71382.65,-0.00
395,31/10/2025,Crédito,A,00008-00008218,CARNIVAL SRL,30676579938,-0.0,-0.00,-0.00,-0.00,-2106948.35
396,31/10/2025,Crédito,A,00008-00008219,CARNIVAL SRL,30676579938,-0.0,-0.00,-0.00,-0.00,-1924116.35
397,31/10/2025,Crédito,A,00008-00008220,CARNIVAL SRL,30676579938,-0.0,-0.00,-0.00,-0.00,-2094700.95


In [6]:
driver.switch_to.frame('marco')

In [7]:

for index, row in clean_data.iterrows():
    # Encuentra cada campo por su id y llena el valor
    driver.find_element(By.ID, 'DetailsView1_TextBox1').send_keys(row['Fecha'])
    driver.find_element(By.ID, 'DetailsView1_TextBox2').send_keys(row['Fecha'])
    driver.find_element(By.NAME, 'DetailsView1$ctl02').send_keys(row['Tipo3'])
    driver.find_element(By.NAME, 'DetailsView1$ctl03').send_keys(row['Factura'])
    driver.find_element(By.NAME, 'DetailsView1$ctl04').send_keys(row['Proveedor'])
    driver.find_element(By.NAME, 'DetailsView1$ctl05').send_keys(row['CUIT'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox3').send_keys(row['NETO 10.5'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox4').send_keys(row['NETO 21'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox5').send_keys(row['IVA 10.5'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox6').send_keys(row['IVA 21'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox7').send_keys(row['TOTAL_NO_GRAVADO'])
    # Haz clic en el botón "Agregar"
    driver.find_element(By.LINK_TEXT, 'Agregar').click()
    # Espera un poco para que la página se actualice
    time.sleep(3)

In [5]:
os.remove(r'C:\Users\ThinkPad-PC\Downloads\Mis Comprobantes Recibidos - CUIT 30714894346.xlsx')